In [1]:
import pandas as pd
import numpy as np  
import re
import sys
import openpyxl

import PyUber

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import PRODUCT as prod
import REDWING as redwing

In [2]:
rw = redwing.RedWing(
    xeus_source='F32_PROD_XEUS',
    debug_flag=True)

In [3]:
NPI = rw.NPIs[2]
match = re.search(r'\((.*?)\)', NPI)
NPI_product = match.group(1) if match else None
NPI_product = NPI_product.replace(' ','%')
# print(NPI_product)

In [4]:
prod1 = prod.Product(npi=NPI,
                product=NPI_product,
                xeus_source = 'F32_PROD_XEUS',
                debug_flag = True)

Running master query for LOT: 'A528013'


d:\Python\NPI\PRODUCT.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['VER','VERSION'], inplace=True)
d:\Python\NPI\PRODUCT.py:301: FutureWarning: The provided callable <function min at 0x0000021894803B00> is currently using DataFrameGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  RetData = pd.pivot_table(RetData,index=['LAYER'], values=['TAPEIN_TREND','ITO_TREND','FAB_REQUIREDDATE','IMO_TREND','SHIPDATE'], aggfunc=np.min).reset_index()


In [5]:
prod1.Reticles

,LAYER,FAB_REQUIREDDATE,IMO_TREND,ITO_TREND,SHIPDATE,TAPEIN_TREND
0,BAV,2025-09-17,2025-09-03,2024-12-23,NaT,2024-12-16
1,BC1,2025-09-18,2025-08-11,2024-12-23,2025-08-11 02:39:04,2024-12-16
2,BC2,2025-09-19,2025-08-13,2024-12-23,2025-08-13 16:38:52,2024-12-16
3,BC3,2025-09-20,2025-08-10,2024-12-23,2025-08-10 10:39:42,2024-12-16
4,BCN,2025-09-08,2025-09-05,2024-12-23,NaT,2024-12-16
...,...,...,...,...,...,...
79,VA8,2025-09-29,2025-08-29,2025-05-01,NaT,2025-04-28
80,VA9,2025-09-30,2025-08-18,2025-05-01,2025-08-18 02:33:27,2025-04-28
81,VCG,2025-08-10,2025-06-17,2024-12-23,2025-06-17 21:43:45,2024-12-16
82,VCT,2025-08-11,2025-06-30,2024-12-23,2025-06-30 22:46:07,2024-12-16


In [6]:
# Future add_lot code def

# rw.NPI_List

lots = rw.get_lots(GROUP_NAME=NPI)

# lots = lots[0:1]

print(lots)
# for lot in lots:
    # print(lot)

# lot_index_dict = {lot: idx for idx, lot in enumerate(lots)}
# print(lot_index_dict)


[{'LOT': 'A5280140', 'LOT_TYPE': 'Bull', 'COMMIT': Timestamp('2025-12-03 18:00:00')}, {'LOT': 'A5289230', 'LOT_TYPE': 'Cross Qual', 'COMMIT': Timestamp('2025-11-15 18:00:00')}, {'LOT': 'A5280130', 'LOT_TYPE': 'Lead Lot', 'COMMIT': Timestamp('2025-11-30 18:00:00')}, {'LOT': 'A5289210', 'LOT_TYPE': 'Scout 1', 'COMMIT': Timestamp('2025-11-26 18:00:00')}, {'LOT': 'A5289220', 'LOT_TYPE': 'Silent Lot', 'COMMIT': Timestamp('2025-11-28 18:00:00')}, {'LOT': 'A528920C', 'LOT_TYPE': 'EF Scout 2', 'COMMIT': Timestamp('2025-11-22 18:00:00')}, {'LOT': 'A528920A', 'LOT_TYPE': 'SC Scout 2', 'COMMIT': Timestamp('2025-11-24 18:00:00')}, {'LOT': 'A5289200', 'LOT_TYPE': 'SC Scout 1', 'COMMIT': Timestamp('2025-11-24 18:00:00')}, {'LOT': 'A528920B', 'LOT_TYPE': 'EF Scout 1', 'COMMIT': Timestamp('2025-11-20 18:00:00')}]


In [7]:
for lot in lots:
    print(f"LOT: {lot['LOT']}, LOT_TYPE: {lot['LOT_TYPE']}, COMMIT: {lot['COMMIT']}")

LOT: A5280140, LOT_TYPE: Bull, COMMIT: 2025-12-03 18:00:00
LOT: A5289230, LOT_TYPE: Cross Qual, COMMIT: 2025-11-15 18:00:00
LOT: A5280130, LOT_TYPE: Lead Lot, COMMIT: 2025-11-30 18:00:00
LOT: A5289210, LOT_TYPE: Scout 1, COMMIT: 2025-11-26 18:00:00
LOT: A5289220, LOT_TYPE: Silent Lot, COMMIT: 2025-11-28 18:00:00
LOT: A528920C, LOT_TYPE: EF Scout 2, COMMIT: 2025-11-22 18:00:00
LOT: A528920A, LOT_TYPE: SC Scout 2, COMMIT: 2025-11-24 18:00:00
LOT: A5289200, LOT_TYPE: SC Scout 1, COMMIT: 2025-11-24 18:00:00
LOT: A528920B, LOT_TYPE: EF Scout 1, COMMIT: 2025-11-20 18:00:00


In [8]:
prod1.load_lot_list(lots)

LOT: A5280140, LOT_TYPE: Bull
Running master query for LOT: 'A5280140'
LOT: A5289230, LOT_TYPE: Cross Qual
Running master query for LOT: 'A5289230'
LOT: A5280130, LOT_TYPE: Lead Lot
Running master query for LOT: 'A5280130'
LOT: A5289210, LOT_TYPE: Scout 1
Running master query for LOT: 'A5289210'
LOT: A5289220, LOT_TYPE: Silent Lot
Running master query for LOT: 'A5289220'
LOT: A528920C, LOT_TYPE: EF Scout 2
Running master query for LOT: 'A528920C'
LOT: A528920A, LOT_TYPE: SC Scout 2
Running master query for LOT: 'A528920A'
LOT: A5289200, LOT_TYPE: SC Scout 1
Running master query for LOT: 'A5289200'
LOT: A528920B, LOT_TYPE: EF Scout 1
Running master query for LOT: 'A528920B'


In [46]:
df = prod1.LotFlows.copy()

In [47]:
lot_start = df['OUT_DATE'].min()
trend_start = lot_start - pd.DateOffset(days=21)
print(f"Lot Start: {lot_start}, Trend Start: {trend_start}")

Lot Start: 2025-07-10 03:58:01, Trend Start: 2025-06-19 03:58:01


In [48]:
def scale_dates(dc, dc_name,start_date):
    dc[f'SCALED_{dc_name}'] = dc[dc_name] - start_date
    dc[f'SCALED_{dc_name}'] = dc[f'SCALED_{dc_name}'].dt.total_seconds() / (24 * 3600)    
    dc[f'SCALED_{dc_name}'] = dc[f'SCALED_{dc_name}'].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
    return dc

In [49]:
df = scale_dates(df, 'OUT_DATE', trend_start)
df = scale_dates(df, 'PLAN', trend_start)


In [60]:
rf = prod1.Reticles.copy()
rf

,LAYER,FAB_REQUIREDDATE,IMO_TREND,ITO_TREND,SHIPDATE,TAPEIN_TREND
0,BAV,2025-09-17,2025-09-03,2024-12-23,NaT,2024-12-16
1,BC1,2025-09-18,2025-08-11,2024-12-23,2025-08-11 02:39:04,2024-12-16
2,BC2,2025-09-19,2025-08-13,2024-12-23,2025-08-13 16:38:52,2024-12-16
3,BC3,2025-09-20,2025-08-10,2024-12-23,2025-08-10 10:39:42,2024-12-16
4,BCN,2025-09-08,2025-09-05,2024-12-23,NaT,2024-12-16
...,...,...,...,...,...,...
79,VA8,2025-09-29,2025-08-29,2025-05-01,NaT,2025-04-28
80,VA9,2025-09-30,2025-08-18,2025-05-01,2025-08-18 02:33:27,2025-04-28
81,VCG,2025-08-10,2025-06-17,2024-12-23,2025-06-17 21:43:45,2024-12-16
82,VCT,2025-08-11,2025-06-30,2024-12-23,2025-06-30 22:46:07,2024-12-16


In [61]:
col_list = ['TAPEIN_TREND','ITO_TREND','FAB_REQUIREDDATE','IMO_TREND','SHIPDATE']
for col in col_list:
    if col not in rf.columns:
        print(f"Column {col} not found in Reticles DataFrame.")
    else:
        rf = scale_dates(rf, col, trend_start)

In [62]:
rf['SCALED_SHIPDATE'] = rf['SCALED_SHIPDATE'] - rf['SCALED_ITO_TREND']
rf['SCALED_IMO_TREND'] = rf['SCALED_IMO_TREND'] - rf['SCALED_ITO_TREND']
rf['SCALED_ITO_TREND'] = rf['SCALED_ITO_TREND'] - rf['SCALED_TAPEIN_TREND']

In [63]:
rf['SCALED_SHIPDATE'] = rf['SCALED_SHIPDATE'].fillna(0)
rf.loc[rf['SCALED_SHIPDATE'] > 0, 'SCALED_IMO_TREND'] = 0

In [64]:
df.to_csv('dump/LotFlows.csv', index=False)
rf.to_csv('dump/Reticles.csv', index=False)